In [185]:
import os
import random
import json
import pandas as pd
import numpy as np
import glob
import math
import numpy
from collections import defaultdict

In [186]:
def getData(path):

    files = glob.glob(path + '*.json', recursive=True)
    vehicle_list, Object_list = [], []
    ObjectCpm = []
    nObjects = 0
    for single_file in files:

        f = open(single_file)
        data = json.load(f)
        vehicle_list.append(data['vehicle'])
        for obj in data['Object']:
            Object_list.append(obj)
        f.close()

    return vehicle_list, Object_list

In [187]:
def getDf(vehicles_list, object_list):
    
    vehicle_df = pd.DataFrame(vehicles_list)
    object_df = pd.DataFrame([object_list[0]])
    nObjects = int(object_list[0]['Id'][1:])
    for obj_dict in object_list:
        value = obj_dict['Id']
        nObjects = max(nObjects, int(obj_dict['Id'][1:]))
        Temp_df = pd.DataFrame()
        Temp_df = object_df.loc[ (object_df['Id'] == value)]
        v = obj_dict['timestamp']
        if(Temp_df.empty):
            object_df = object_df.append(obj_dict,ignore_index=True)

        else:
            if(Temp_df['timestamp'].iloc[0] < v):
                object_df.loc[object_df['Id'] == obj_dict['Id']] = [pd.Series(obj_dict)]
            else:
                continue

    return vehicle_df, object_df, nObjects

In [188]:
def loadCalculator(optimisedSet, coverage, object_df):
    load = 0
    objectSize = {'person':15, 'obstruction': 20}
    for x in optimisedSet:
        i = int(x[1])
        for col in range(len(coverage[i])):
            if coverage[i][col]:
                load += objectSize[object_df.loc[object_df['Id'] == str('o'+ str(i))]['type'].values[0]]
    return load

In [189]:
def calcObjectCoverage(vehicle_df, object_df, nObjects):
    vehicles, objects = vehicle_df.shape[0], nObjects+1
    coverage = []
    
    for v in range(vehicles):
        temp_cov = []
        temp_vdf = vehicle_df.loc[vehicle_df['Id'] == str('v' + str(v))]
        
        V_xcoord = int(temp_vdf['x_coordinate'].values[0])
        V_ycoord = int(temp_vdf['y_coordinate'].values[0])
        V_radius = float(temp_vdf['radius'].values[0])

        for o in range(objects):
            temp_odf = object_df.loc[object_df['Id'] == str('o' + str(o))]
            
            if temp_odf.empty:
                temp_cov.append(False)
                continue

            O_xcoord = int(temp_odf['x_coordinate'].values[0])
            O_ycoord = int(temp_odf['y_coordinate'].values[0])
    
            euclidDist = math.sqrt(math.pow((V_xcoord - O_xcoord),2) + math.pow((V_ycoord - O_ycoord),2))

            if(euclidDist < (V_radius)):
                temp_cov.append(True)
            else:
                temp_cov.append(False)

        coverage.append(temp_cov)
    
    return coverage

In [190]:
#Submodular function
def NoOfObjects(vehicle_df, object_df, coverage):
    leftVehicle = defaultdict(set)
    for i in range(len(vehicle_df)):
        key = vehicle_df.loc[i, "Id"]
        ind = int(str(key)[1:])
        vSet = set()
        for i in range(len(coverage[ind])):
            if coverage[ind][i]:
                vSet.add(str('o'+str(i)))
        leftVehicle[key] = vSet
    return leftVehicle

In [191]:
#Marginal Gain
def getMarginalGain(objectSet, leftVehicle, delta):
    maxSize = delta-1
    selectedV = None
    for v in leftVehicle:
        s = len(leftVehicle[v].difference(objectSet))
        if s > maxSize:
            maxSize = s
            selectedV = v
    if maxSize <= delta:
        return objectSet, leftVehicle, None
    else:
        objectSet = objectSet.union(leftVehicle[selectedV])
        leftVehicle.pop(selectedV)
        return objectSet, leftVehicle, selectedV

In [192]:
#Greedy Appoximation
def greedyAppx(leftVehicle, delta):
    optimisedSet = set()
    objectSet = set()
    flag = True
    while flag:
        objectSet, leftVehicle, v = getMarginalGain(objectSet, leftVehicle, delta)
        if not v:
            flag = False
        else:
            optimisedSet.add(v)
    return optimisedSet, objectSet